In [14]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import comms
import ipywidgets as ipw

import keras 
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

## Load the data 

In [2]:
data_train = comms.load_data("train.json/data/processed/train.json")

In [38]:
data_train = data_train[data_train.inc_angle != "na"]

In [39]:
data_train.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [41]:
ipw.interact(lambda i : comms.plot_img(data = data_train, band = 0, index = i), 
             i = (0, data_train.shape[0]-1, 1)
            )

interactive(children=(IntSlider(value=735, description='i', max=1470), Output()), _dom_classes=('widget-interact',))

<function __main__.<lambda>>

## Design a model in Keras

In [5]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(75, 75, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 32)        608       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 35, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 31, 31, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 128)       73856     
__________

## Train the model 

In [7]:
img_train, ang_train = comms.get_img_and_angle(data_train)
is_ice_train = comms.get_is_iceberg(data_train)

In [8]:
model.fit(img_train, is_ice_train, epochs = 5)

Epoch 1/5
1604/1604 [==============================] - 63s - loss: 0.6797    
Epoch 2/5
1604/1604 [==============================] - 63s - loss: 0.5130    
Epoch 3/5
1604/1604 [==============================] - 67s - loss: 0.5302    
Epoch 4/5
1604/1604 [==============================] - 69s - loss: 0.4666    
Epoch 5/5
1604/1604 [==============================] - 67s - loss: 0.4670    


## Make predictions for the test data 

In [9]:
data_test = comms.load_data("test.json/data/processed/test.json")

In [10]:
img_test, ang_test = comms.get_img_and_angle(data_test)

In [11]:
is_ice_test = model.predict(img_test)

In [12]:
id_test = np.array(data_test.id)

In [13]:
res = pd.DataFrame({"id" : id_test.flatten(), "is_iceberg" : is_ice_test.flatten()})
res.to_csv("result.csv", sep=",", index=False)